In [3]:
import torch
import json
from tqdm import tqdm
import random


ENTITY_FEATURES_FILE = '../data/wikidata5m_inductive/entity_description_first_sentence_embedding.pt'
ENTITY_2_ID_FILE = '../data/wikidata5m_inductive/uri_to_id.json'
RELATION_2_ID_FILE = '../data/wikidata5m_inductive/relation_uri_to_id.json'

In [2]:
with open(ENTITY_2_ID_FILE) as uri_to_id_in:
    uri_to_id = json.load(uri_to_id_in)

with open(RELATION_2_ID_FILE) as uri_to_id_in:
    relation_uri_to_id = json.load(uri_to_id_in)

In [11]:
all_uris = list(uri_to_id.keys())
random.shuffle(all_uris)
sampled_uris = set(all_uris[:500000])

In [14]:
for sample in ['train', 'valid', 'test']:
    sampled_triples = []
    with open(f'../data/wikidata5m_inductive/wikidata5m_inductive_{sample}.txt') as triples_in:
            for line in triples_in:
                head, relation, tail = line[:-1].split('\t')
                if head in sampled_uris and tail in sampled_uris:
                    sampled_triples.append([head, relation, tail])

    with open(f'../data/wikidata5m_inductive/wikidata5m_inductive_exp_{sample}.txt', 'w') as triples_out:
        for triple in sampled_triples:
            triples_out.write('\t'.join(triple) + '\n')

In [ ]:
def read_lp_data(file_path, uri_to_id, relation_uri_to_id, data_sample):
    print('Preprocessing:', data_sample)

    edge_index = []
    edge_type = []
    with open(file_path) as triples_in:
        for line in triples_in:
            head, relation, tail = line[:-1].split('\t')
            edge_index.append([uri_to_id[head], uri_to_id[tail]])
            edge_type.append(relation_uri_to_id[relation])

    return Data(edge_index=torch.tensor(edge_index).t(),
                edge_type=torch.tensor(edge_type))